In [ ]:
import pandas as pd
import psycopg2
import random
import numpy as np
from datetime import datetime, timedelta

In [ ]:
#Leemos el CSV generado con datos aleatorios antes
df_hospitalPatientRecords_IDEA4RC = pd.read_csv("./IDEA4RC-data/hospitalPatientsRecordIDEA4RC.csv")
df_hospitalPatientRecords_IDEA4RC.head(5)

Ahora vamos a querer comprobar que la conexión se estableció correctamente

In [ ]:
conn_params = {
    'dbname': 'omopdb',
    'user': 'postgres',
    'password': 'mysecretpassword',
    'host': 'omop_postgres',
    'port': '5432'
}

try:
    # Conexión a la base de datos
    conn = psycopag2.connect(**conn_params)
    cur = conn.cursor()
    
    # Verifica la base de datos actual
    cur.execute("SELECT current_database();")
    current_db = cur.fetchone()[0]
    print("Base de datos actual:", current_db)
    
    # Verifica el esquema y tablas disponibles
    cur.execute("SELECT table_schema, table_name FROM information_schema.tables WHERE table_name = 'visit_occurrence';")
    tables = cur.fetchall()
    if not tables:
        print("La tabla 'care_site' no existe.")
    else:
        print("Tabla 'care_site' encontrada en los siguientes esquemas:", tables)
    
except Exception as e:
    print("Ocurrió un error:", e)

In [ ]:
df_table = pd.DataFrame()
df_table.index = range(len(df_hospitalPatientRecords_IDEA4RC))
sql = """
    INSERT INTO public.visit_occurence (visit_occurrence_id,person_id,care_site_id,visit_start_date)
    VALUES (%s,%s,%s,%s)
"""
cur.executemany(sql, zip(df_hospitalPatientRecords_IDEA4RC['id'],df_hospitalPatientRecords_IDEA4RC['Patient'],
                         df_hospitalPatientRecords_IDEA4RC['Hospital'],df_hospitalPatientRecords_IDEA4RC['Date of first contact with the hospital']))
conn.commit()
#Partiendo de todo lo que está ya hecho, esto debería de tener sentido.